In [2]:
# Load Libraries
import pandas as pd

In [3]:
# Viếng thăm

# Define the file path
file_path = 'Dataset/dms.visit.partner_visit.xlsx'

# Load the Excel file
df_visit = pd.read_excel(file_path)

# Group by 'Nhân viên viếng thăm' and count the number of visits
visit_counts = df_visit.groupby('Nhân viên viếng thăm').size().reset_index(name='Tổng số lượt đi')

# Display the result
visit_counts


,Nhân viên viếng thăm,Tổng số lượt đi
0,MT- MEKONG1,33
1,MT- MIENDONG,69
2,MT- MIENTRUNG,47
3,MT-QLKDMN,22
4,MT-QLKDMN3,1
5,MTHCM-KV1,85
6,MTHCM-KV2,92
7,MTHCM-KV3,105
8,MTHCM-KV5,102
9,MTHCM-KV6,75


In [48]:
# Độ phủ

# 
# Define the file paths
file_path_fill = 'Dataset/dms.visit.partner_fill.xlsx'

file_path_customer = 'Dataset/customer.xlsx'  # Update this to the correct path for df_customer

# Load the Excel file -> fill dow
df_fill= pd.read_excel(file_path_fill)


df_fill=df_fill.fillna(method='ffill')
df_fill['Ngày tạo']=df_fill['Ngày tạo'].ffill(axis=0)

#drop Column
# df_fillmerged=df_fillmerged.drop(['Tuyến/Tên', 
#          'Chi tiết kiểm tồn/Sample Quantity',
#          'Chi tiết kiểm tồn/Hạn sử dụng',
#          'Mã','Đăng nhập','Đăng xuất'], axis=1)

# # Drop null values in `Ngày tạo`
# df_fillmerged.dropna(subset=['Ngày tạo'], inplace=True)
# df_fillmerged.head()

C:\Users\Nguyen Vu\AppData\Local\Temp\ipykernel_16300\2251225673.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_fill=df_fill.fillna(method='ffill')


In [49]:
# WRITE EXCEL
file_name = 'Dataset/df_fillmerged.xlsx'
dataset_write=df_merged
 
# creating an ExcelWriter object
with pd.ExcelWriter(file_name) as writer:
    dataset_write.to_excel(writer, sheet_name='Data', index=False)

In [ ]:
# CUSTOMER -> MERGED
# Khách hàng
df_customer=pd.read_excel(file_path_customer,sheet_name='Data khách hàng')

# Merge df_fill with df_customer to get 'Nhân viên KD' and 'Loại hình'
df_merged = pd.merge(df_fill, df_customer[['Mã', 'Nhân viên KD', 'Loại hình']], left_on='Khách hàng/Mã', right_on='Mã', how='left')

df_merged['Ngày tạo'] = pd.to_datetime(df_visit['Ngày tạo'])


# Bỏ quản lý
list_quan_ly = ['MT-QLKDMN','MT-QLKDMN3']

df_fillmerged=df_merged[-df_merged['Nhân viên viếng thăm'].isin(list_quan_ly)].rename(columns={
    'Chi tiết kiểm tồn/Sản phẩm/Mã nội bộ': 'Mã sản phẩm',
    'Khách hàng/Mã': 'Mã khách hàng',
    'Khách hàng' : 'Mã khách hàng ERP',
    'Khách hàng/Tên':'Tên khách hàng',
    'Chi tiết kiểm tồn/Sản phẩm/Tên': 'Tên sản phẩm',
    'Chi tiết kiểm tồn/Số lượng chẵn': 'Tồn',
    'Chi tiết kiểm tồn/Sản phẩm/Đơn vị tính':'Đơn vị tính'
})

In [6]:
# Group by the specified columns and calculate the sum of `Tồn`
df_fill_store = df_fillmerged.groupby(['Mã khách hàng', 'Mã sản phẩm', 'Ngày tạo'])['Tồn'].sum().reset_index()


# Merging back to get additional columns
columns_to_merge = ['Tên khách hàng', 'Tên sản phẩm', 'Nhân viên KD', 'Loại hình', 'Đơn vị tính']

df_fill_store = df_fill_store.merge(
    df_fillmerged[['Mã khách hàng', 'Mã sản phẩm', 'Ngày tạo'] + columns_to_merge],
    on=['Mã khách hàng', 'Mã sản phẩm', 'Ngày tạo'],
    how='left',
)

# Sorting the final DataFrame (optional)
df_final_df = df_fill_store.sort_values(by=['Ngày tạo','Mã khách hàng', 'Mã sản phẩm'])
df_fill_store.head()

,Mã khách hàng,Mã sản phẩm,Ngày tạo,Tồn,Tên khách hàng,Tên sản phẩm,Nhân viên KD,Loại hình,Đơn vị tính
0,002MT BDG WM+4096,CACOM23088,2024-07-06,50.0,WM+ BDG 14A ĐT 743,Cá cơm vị cay ngọt ALACO 12g,MT- MIENDONG,Win+,Gói
1,002MT BDG WM+4096,CACOM23095,2024-07-06,50.0,WM+ BDG 14A ĐT 743,Cá cơm vị truyền thống ALACO 12g,MT- MIENDONG,Win+,Gói
2,002MT BDG WM+4096,RBGW20018,2024-07-06,10.0,WM+ BDG 14A ĐT 743,Rong biển Gimbab (10 lá) 20g,MT- MIENDONG,Win+,Gói
3,002MT BDG WM+4096,TRCHAU72226,2024-07-06,2.0,WM+ BDG 14A ĐT 743,Trân châu kim cương Oppa Bubble hộp 350g ( 35g...,MT- MIENDONG,Win+,hộp
4,002MT BDG WM+4228,BGYP00192,2024-07-06,1.0,"WM+ BDG Thửa 4128, KP Nội Hoá 2",Yopokki Bánh Gạo Topokki vị phomai 120g,MT- MIENDONG,Win+,Cốc


In [7]:

# Step 1: Group by 'Mã khách hàng', 'Mã sản phẩm', 'Ngày tạo' and calculate the sum of 'Tồn'
df_grouped = df_fillmerged.groupby(['Mã khách hàng', 'Mã sản phẩm', 'Ngày tạo']).agg({'Tồn': 'sum'}).reset_index()

# Step 2: Add 'Check_Show' and 'Product_Exist' columns based on the 'Tồn' value
df_grouped['Check_Show'] = df_grouped['Tồn'].apply(lambda x: 1 if x > 0 else 0)
df_grouped['Product_Exist'] = 1

df_merged = df_grouped.merge(df_fillmerged[['Mã khách hàng', 'Mã sản phẩm', 'Nhân viên KD', 'Loại hình']], on=['Mã khách hàng', 'Mã sản phẩm'], how='left').drop_duplicates()

# Group by 'Nhân viên KD', 'Loại hình' and calculate the sum of 'Check_Show' and 'Product_Exist'
df_pivot = df_merged.groupby(['Nhân viên KD', 'Loại hình']).agg({'Check_Show': 'sum', 'Product_Exist': 'sum'}).reset_index()

# Calculate 'Fill_Store_percent' as Sum(Check_Show) / Sum(Product_Exist) * 100
df_pivot['Fill_Store_percent'] = (df_pivot['Check_Show'] / df_pivot['Product_Exist']) * 100

# Create the pivot table
pivot_fill_store = df_pivot.pivot_table(index='Nhân viên KD', columns='Loại hình', values='Fill_Store_percent').reset_index()

pivot_fill_store

Loại hình,Nhân viên KD,Siêu Thị,Win+,Đại Siêu Thị
0,MT- MEKONG,NaN,NaN,37.662338
1,MT- MIENDONG,100.000000,100.0,NaN
2,MTHCM-KV2,20.000000,NaN,NaN
3,MTHCM-KV3,23.232323,NaN,62.962963
4,MTHCM-KV5,25.641026,NaN,NaN
5,MTHCM-KV6,39.393939,NaN,52.325581


In [210]:
df_merged

,Mã khách hàng,Mã sản phẩm,Ngày tạo,Tồn,Check_Show,Product_Exist,Nhân viên KD,Loại hình
0,002MT BDG WM+4096,CACOM23088,2024-07-06,50.0,1,1,MT- MIENDONG,Win+
1,002MT BDG WM+4096,CACOM23095,2024-07-06,50.0,1,1,MT- MIENDONG,Win+
2,002MT BDG WM+4096,RBGW20018,2024-07-06,10.0,1,1,MT- MIENDONG,Win+
3,002MT BDG WM+4096,TRCHAU72226,2024-07-06,2.0,1,1,MT- MIENDONG,Win+
4,002MT BDG WM+4228,BGYP00192,2024-07-06,1.0,1,1,MT- MIENDONG,Win+
...,...,...,...,...,...,...,...,...
675,002MT VLG WM1595,TPBS00381,2024-07-15,0.0,0,1,MT- MEKONG,Đại Siêu Thị
676,002MT VLG WM1595,TRCHAU72219,2024-07-13,5.0,1,1,MT- MEKONG,Đại Siêu Thị
678,002MT VLG WM1595,TRCHAU72219,2024-07-15,0.0,0,1,MT- MEKONG,Đại Siêu Thị
680,002MT VLG WM1595,TRCHAU72226,2024-07-13,4.0,1,1,MT- MEKONG,Đại Siêu Thị


In [8]:
# WRITE TO EXCEL:  determining the name of the file
file_name = 'Dataset/Export.xlsx'

dataset_write = df_merged

# creating an ExcelWriter object
with pd.ExcelWriter(file_name) as writer:
    # writing to the 'Employee' sheet
    dataset_write.to_excel(writer, sheet_name='Data', index=False)

In [14]:
df_merged.columns

Index(['Nhân viên viếng thăm', 'Khách hàng', 'Ngày tạo', 'Tuyến/Tên',
       'Địa chiểm checkin', 'Đăng nhập', 'Đăng xuất', 'Khách hàng/Mã',
       'Khách hàng/Tên', 'Chi tiết kiểm tồn/Sản phẩm/Mã nội bộ',
       'Chi tiết kiểm tồn/Sản phẩm/Tên',
       'Chi tiết kiểm tồn/Sản phẩm/Đơn vị tính',
       'Chi tiết kiểm tồn/Số lượng chẵn', 'Chi tiết kiểm tồn/Sample Quantity',
       'Chi tiết kiểm tồn/Hạn sử dụng', 'Mã', 'Nhân viên KD', 'Loại hình'],
      dtype='object')